We have to import the python libraries we need for the Demo Azure open Ai assistant

We will use the openai libray for access, os, json, dotenv for loading secrets key in secrets.env 

- openai
- dotenv

In [4]:
import os
import time
import json
from openai import AzureOpenAI
import dotenv

ImportError: cannot import name 'AzureOpenAI' from 'openai' (C:\Users\olmertens\AppData\Roaming\Python\Python311\site-packages\openai\__init__.py)

In [ ]:
dotenv.load_dotenv(./secrets.env)
azure_openaikey = os.getenv("AZURE_OPENAIKEY")
azure_openaiendpoint = os.getenv("AZURE_OPENAIENDPOINT")
azure_openaideployment = os.getenv("AZURE_OPENAIDEPLOYMENT")

We could now instantiate the AzurepeneAi client

In [ ]:
client = AzureOpenAI(
    api_key= azure_openaikey
    api_version="2024-02-15-preview",
    azure_endpoint = azure_openaiendpoint
    )

### Instantiate the client from the client AzureOpenAi Client

In [ ]:

# Create an assistant
assistant = client.beta.assistants.create(
    name="Math Assist",
    instructions="You are an AI assistant that can write code to help answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model=azure_openaideployment #You must replace this value with the deployment name for your model.
)


In [ ]:

# Create a thread
thread = client.beta.threads.create()

In [ ]:

# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)

In [ ]:

# Run the thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

In [ ]:

# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

status = run.status

In [ ]:

# Wait till the assistant has responded
while status not in ["completed", "cancelled", "expired", "failed"]:
    time.sleep(5)
    run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
    status = run.status

messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [ ]:

print(messages.model_dump_json(indent=2))